In [17]:
import sys
import time
import json
sys.path.insert(0, 'src/data')
sys.path.insert(0, 'src/dgl_graphsage')

import recommend as r
from train_updated import train
from src.api.spotifyAPI import SpotifyAPI

from sklearn.neighbors import NearestNeighbors
import pandas as pd

In [2]:
%%time
# Loading Graph ~ 3min
feat_data, adj_list, dgl_G, uri_map = r.load_double_edge("./data/a13group1/features/merged_features.csv")

Loading feature data...
Loaded DGL Graph
adj list created
CPU times: user 2min 38s, sys: 6.91 s, total: 2min 45s
Wall time: 2min 45s


In [3]:
%%time
# Training the Model. GPU ~ 00:00:40
with open('config/model-params.json') as fh:
            model_cfg = json.load(fh)
model, pred = train(dgl_G, feat_data, adj_list, **model_cfg)

# Put everything on CPU
model = model.to('cpu')
pred = pred.to('cpu')

Train pos edge: 26413580
Validation pos edge: 518464
Cuda enabled: True

Training starts:
In epoch 1 batch 1, loss: 0.7617809176445007
In epoch 1 batch 6, loss: 0.6285180449485779
In epoch 1 batch 11, loss: 0.38366028666496277
In epoch 1 batch 16, loss: 0.1346568912267685
In epoch 1 batch 21, loss: 0.11267966032028198

Epoch 1 AUC:  0.4559053349872587
In epoch 2 batch 1, loss: 0.14307011663913727
In epoch 2 batch 6, loss: 0.15115752816200256
In epoch 2 batch 11, loss: 0.13045252859592438
In epoch 2 batch 16, loss: 0.11427348107099533
In epoch 2 batch 21, loss: 0.10287398099899292

Epoch 2 AUC:  0.49749590200129074
In epoch 3 batch 1, loss: 0.11056103557348251
In epoch 3 batch 6, loss: 0.11047366261482239
In epoch 3 batch 11, loss: 0.1061316579580307
In epoch 3 batch 16, loss: 0.10686339437961578
In epoch 3 batch 21, loss: 0.11026901751756668

Epoch 3 AUC:  0.528041579923877
In epoch 4 batch 1, loss: 0.10805072635412216
In epoch 4 batch 6, loss: 0.10907457023859024
In epoch 4 batch 11, 

In [4]:
# Create Embeddings
z = model(dgl_G, feat_data)

# Create Predictions
# In the form of the strength of the connection between source, destination from dgl_G.edges()
preds = pred(dgl_G, z)

In [5]:
# Create Nearest Neighbors
neigh = NearestNeighbors(n_neighbors=25, radius=0.4)
neigh.fit(feat_data)

NearestNeighbors(n_neighbors=25, radius=0.4)

In [6]:
# Get a random playlist
item = r.get_random_playlist()

/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.7000-7999.json


In [7]:
# Get playlist information
item, original_tracks, seeds = r.get_playlist_info(item)
original_tracks

Playlist ID: 7009
Playlist Length: 20


['Sneakin’---Drake',
 'Gassed Up---Nebu Kiniza',
 'You Was Right---Lil Uzi Vert',
 'Fire Squad - Live---J. Cole',
 'A Tale of 2 Citiez - Live---J. Cole',
 'Deja Vu---J. Cole',
 '20 Joints---Berner',
 'Moves---Big Sean',
 'Look At Me!---XXXTENTACION',
 'Rent Money---Future',
 'No Heart---21 Savage',
 'Dear Mama---2Pac',
 'Neighbors---J. Cole',
 '4 Your Eyez Only---J. Cole',
 'Miss America---J. Cole',
 'Like Toy Soldiers---Eminem',
 'T-Shirt---Migos',
 'Get Your Walk On---Xzibit',
 'X---Xzibit',
 'I Need A Doctor---Dr. Dre']

In [8]:
# Get recommendations
uri_recs = r.recommend(seeds, dgl_G, z, pred, neigh, feat_data, uri_map)
uri_recs

['3hxsFtkwEx6rZ310ROjwPD',
 '7nQFMZ3IbTFeIA8RxRnzBy',
 '6B0FIZm6Xs4qYRyTjsuU0Y',
 '6jalstnZd8hEJuLGFFjL9L',
 '6EaJ6rsAqrcfsEhKzrtK9i',
 '7dEYcnW1YSBpiKofefCFCf',
 '1eLU27PPFxBhbp1CBjPh9J',
 '3hxsFtkwEx6rZ310ROjwPD',
 '08azaGQ1YKWvajMDARPSeZ',
 '7dEYcnW1YSBpiKofefCFCf',
 '5hzrBGIoRlsein6TvuahVd',
 '0pSaDwrMLM7F82LWJg9v6s',
 '4sHEykAZCmyugZQBJZ2GKu',
 '5XlpERwWF8SuVUL9AghEIh',
 '7bFElqzAlHXnnINRBXRQbK',
 '7mcPcohfR96oIGbv6dxS3Q',
 '3hxsFtkwEx6rZ310ROjwPD',
 '5R5zty5rJHUs5nbpKjzrrD',
 '7crrp07AQujRvbOoIGitk8',
 '6B0FIZm6Xs4qYRyTjsuU0Y']

In [12]:
# Translate to Song Names
client_id = ''
client_secret = ''

spotify = SpotifyAPI(client_id, client_secret)

In [13]:
%%time
rec_track_names = r.get_rec_names(uri_recs, spotify, 2)

CPU times: user 548 ms, sys: 66.9 ms, total: 615 ms
Wall time: 44.1 s


In [14]:
original_tracks

['Sneakin’---Drake',
 'Gassed Up---Nebu Kiniza',
 'You Was Right---Lil Uzi Vert',
 'Fire Squad - Live---J. Cole',
 'A Tale of 2 Citiez - Live---J. Cole',
 'Deja Vu---J. Cole',
 '20 Joints---Berner',
 'Moves---Big Sean',
 'Look At Me!---XXXTENTACION',
 'Rent Money---Future',
 'No Heart---21 Savage',
 'Dear Mama---2Pac',
 'Neighbors---J. Cole',
 '4 Your Eyez Only---J. Cole',
 'Miss America---J. Cole',
 'Like Toy Soldiers---Eminem',
 'T-Shirt---Migos',
 'Get Your Walk On---Xzibit',
 'X---Xzibit',
 'I Need A Doctor---Dr. Dre']

In [18]:
pd.Series(rec_track_names).value_counts()

Got It (feat. Snoop Dogg)---Boys Noize                                                                           3
Bad Meets Evil---Eminem                                                                                          2
Ka-Bang! (feat. MF Doom)---CZARFACE                                                                              2
Just Life---Mark Battles                                                                                         1
Inertia---ARTY                                                                                                   1
Love Yourz - Live---J. Cole                                                                                      1
1Train (feat. Kendrick Lamar, Joey Bada$$, Yelawolf, Danny Brown, Action Bronson & Big K.R.I.T.)---A$AP Rocky    1
Rap Up 2016---Uncle Murda                                                                                        1
Do Your Time---Ludacris                                                         

Note: We kept repeats because a song that is recommended as the best more than once throughout the playlist should be ranked higher